In [1]:
from PIL import Image
import struct

In [37]:
def getpixelByBitmapWay(imgp, size, x, y):
    pix = imgp[x, size-y-1]
    if pix[3] == 0:
        pix = (0,0,0,0)
    return pix

def generateMask(imgSize, imgp):
    bitmapImgMask = bytes()
    for y in range(imgSize):
        for x in range(0, imgSize, 8):
            pix = [getpixelByBitmapWay(imgp, imgSize, _i, y)[3] for _i in range(x,x+8)]
            pix = sum(pix)//8
            bitmapImgMask += struct.pack(
                r'B', pix
            )
    return bitmapImgMask

def imageToBitmap(img):
    if img.size[0] != img.size[1]:
        raise ValueError('img.size %s'%(img.size))
    
    imgSize = img.size[0]
    img = img.convert('RGBA')
    imgp = img.load()
    
    bitmapInfoHeader = struct.pack(
        r'IllHHIIllII',
        40, imgSize, imgSize*2,
        1, 32, 0, imgSize**2*4,
        0, 0, 0, 0
    )
    
    bitmapImgData = bytes()
    for y in range(imgSize):
        for x in range(imgSize):
            bitmapImgData += struct.pack(
                r'BBBB',
                *getpixelByBitmapWay(imgp, imgSize, x, y)
            )

    bitmapImgMask = generateMask(imgSize, imgp)
    
    return bitmapInfoHeader + bitmapImgData + bitmapImgMask

In [43]:
def generateIcon(ipath, size_list=[16,32,64,128,256]):
    for e in size_list:
        if e <= 0 or e > 256:
            raise ValueError('size_list %s'%e)
    size_list.sort()
    
    imgraw = Image.open(ipath)

    if imgraw.size[0] != imgraw.size[1]:
        raise ValueError('img.size %s'%(img.size))
        
    
    iconHeader = struct.pack(
        r'HHH',
        0, 1, len(size_list)
    )
    
    bitmapDatas = []
    for nsize in size_list:
        img = imgraw.resize((nsize,nsize), Image.ANTIALIAS)
        bitmapDatas.append(imageToBitmap(img))
    
    iconEntries = []
    ptr = 6 + len(size_list)*16
    for nsize, bitmapdata in zip(size_list, bitmapDatas):
        if nsize == 256:
            nsize = 0
        datalen = len(bitmapdata)
        iconEntries.append(
            struct.pack(
                r'BBBBHHII',
                nsize, nsize, 0, 0,
                1, 32, datalen, ptr
            )
        )
        ptr += datalen
    
    fileBytes = iconHeader
    for e in iconEntries:
        fileBytes += e
    for e in bitmapDatas:
        fileBytes += e
    
    return fileBytes

In [45]:
with open('./test.ico','wb') as f:
    f.write(generateIcon('./icon.png'))